In [1]:
from random import shuffle
import numpy as np
import glob
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import PIL.Image as Image

In [2]:
#dataset_name = ["text_sample_eng"]     # Single
dataset_name = [                # Multiple
    "dataset_1",
    "dataset_2",
    "dataset_3"
]

In [3]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [4]:
def encode_utf8_string(text, length, word_dict, null_char_id=0):
    #char_ids_padded = [null_char_id]*length
    char_ids_unpadded = [null_char_id]*len(text)
    for i in range(len(text)):
        hash_id = word_dict[text[i]]
        #char_ids_padded[i] = hash_id
        char_ids_unpadded[i] = hash_id
    #return char_ids_padded, char_ids_unpadded
    return char_ids_unpadded

In [5]:
addrs_image, addrs_label = [], []
for ds_name in dataset_name:
    image_path = 'data/{}/images_padded/*.png'.format(ds_name)  # resized
    label_path = 'data/{}/labels/*.txt'.format(ds_name)
    addrs_image.extend(glob.glob(image_path))
    addrs_label.extend(glob.glob(label_path))

print(len(addrs_image))
print(len(addrs_label))

121
121


In [6]:
# Note that \n and " " must be manually rewritten as <newline>, <whitespace> to avoid mis-decoding
word_dict = dict()
word_dict["<nul>"] = 0
word_dict["\n"] = 1
word_dict[" "] = 2

In [7]:
with open('word_bag_3.txt', encoding="utf-8") as dict_file:
    for line in dict_file:
        (key, value) = line.strip().split('\t')
        word_dict[value] = int(key)
print(len(word_dict))

98


In [8]:
# Decode Example
inv_map = {v: k for k, v in word_dict.items()}

for text in open(addrs_label[0], encoding="utf-8"):
    #char_ids_padded, char_ids_unpadded = encode_utf8_string(
    char_ids_unpadded = encode_utf8_string(
        text=text, 
        word_dict=word_dict,
        length=32)
    print(char_ids_unpadded)
    print(str([inv_map[k] for k in char_ids_unpadded]))
    break

[83, 66, 64, 52, 2, 18, 2, 42, 83, 75, 1]
['장', '비', '분', '류', ' ', ':', ' ', '눈', '장', '식', '\n']


In [20]:
tfrecord_writer = tf.io.TFRecordWriter("data/tfexample_train")

In [21]:
for j in range(0, int(len(addrs_image))): 
#for j in range(0, 1): 
    print('Train data: {}/{}'.format(j, int(len(addrs_image)) -1))
    img = Image.open(addrs_image[j])
    img = img.convert("RGB")        # 

    np_data = np.array(img)
    image = tf.image.convert_image_dtype(np_data, dtype=tf.uint8)
    encoded_image = tf.io.encode_png(image)

    text, char_ids_padded, char_ids_unpadded = [], [], []   
    for txt in open(addrs_label[j], encoding="utf-8"):
        unpadded = encode_utf8_string(
            text=txt,
            word_dict=word_dict,
            length=32)
        char_ids_unpadded.extend(unpadded)
        text.extend(txt)
    text = "".join(text)
    char_ids_padded.extend(char_ids_unpadded)
    while len(char_ids_padded) != 320:      # Padded Length = 32 * 10 = 320
        char_ids_padded.append(0)
    #print(text)
    #print(len(char_ids_unpadded))
    #print(len(char_ids_padded))
    #print("-------------------------------")

    example = tf.train.Example(features=tf.train.Features(
            feature={
                'image/encoded': _bytes_feature(encoded_image),
                'image/format': _bytes_feature(b"PNG"),
                'image/width': _int64_feature([np_data.shape[1]]),
                'image/orig_width': _int64_feature([np_data.shape[1]]),
                'image/class': _int64_feature(char_ids_padded),
                'image/unpadded_class': _int64_feature(char_ids_unpadded),
                'image/text': _bytes_feature(bytes(text, 'utf-8')),                
                ## 'height': _int64_feature([crop_data.shape[0]]),
            }
        ))    
    tfrecord_writer.write(example.SerializeToString())

Train data: 0/120
Train data: 1/120
Train data: 2/120
Train data: 3/120
Train data: 4/120
Train data: 5/120
Train data: 6/120
Train data: 7/120
Train data: 8/120
Train data: 9/120
Train data: 10/120
Train data: 11/120
Train data: 12/120
Train data: 13/120
Train data: 14/120
Train data: 15/120
Train data: 16/120
Train data: 17/120
Train data: 18/120
Train data: 19/120
Train data: 20/120
Train data: 21/120
Train data: 22/120
Train data: 23/120
Train data: 24/120
Train data: 25/120
Train data: 26/120
Train data: 27/120
Train data: 28/120
Train data: 29/120
Train data: 30/120
Train data: 31/120
Train data: 32/120
Train data: 33/120
Train data: 34/120
Train data: 35/120
Train data: 36/120
Train data: 37/120
Train data: 38/120
Train data: 39/120
Train data: 40/120
Train data: 41/120
Train data: 42/120
Train data: 43/120
Train data: 44/120
Train data: 45/120
Train data: 46/120
Train data: 47/120
Train data: 48/120
Train data: 49/120
Train data: 50/120
Train data: 51/120
Train data: 52/120
Tra

In [17]:
bytes(text, 'utf-8').decode("utf-8")

&#39;장비분류 : 눈장식\nSTR : +21 (6 +15)\nDEX : +72 (6 +66)\nINT : +36 (6 +30)\nLUK : +21 (6 +15)\n공격력 : +1\n마력 : +1\n방어력 : +100\n업그레이드 가능 횟수: 3 (복구 가능 횟수: 0)\n&#39;

In [34]:
bytes(text, 'utf-8')

b&#39;\xec\x9e\xa5\xeb\xb9\x84\xeb\xb6\x84\xeb\xa5\x98 : \xeb\x88\x88\xec\x9e\xa5\xec\x8b\x9d\nSTR : +21 (6 +15)\nDEX : +72 (6 +66)\nINT : +36 (6 +30)\nLUK : +21 (6 +15)\n\xea\xb3\xb5\xea\xb2\xa9\xeb\xa0\xa5 : +1\n\xeb\xa7\x88\xeb\xa0\xa5 : +1\n\xeb\xb0\xa9\xec\x96\xb4\xeb\xa0\xa5 : +100\n\xec\x97\x85\xea\xb7\xb8\xeb\xa0\x88\xec\x9d\xb4\xeb\x93\x9c \xea\xb0\x80\xeb\x8a\xa5 \xed\x9a\x9f\xec\x88\x98: 3 (\xeb\xb3\xb5\xea\xb5\xac \xea\xb0\x80\xeb\x8a\xa5 \xed\x9a\x9f\xec\x88\x98: 0)\n&#39;

In [22]:
for raw_record in tf.data.TFRecordDataset("data/tfexample_train"):
#for raw_record in tf.data.TFRecordDataset("fsns-00000-of-00001"):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    #print(example.features.feature["image/text"])
    print(example.features)
    break

\371\347\237+\225\312\232\232\032,\332\313\267\244\001\300\222%K***\016f\331\233={\366\365\327__p\360\236{\356Y\266l\331\301\0060m\332\264\371\363\347\027\034\334\272uk[[\033\252\254o\277\3756\000\250\325\352[o\275u\377\376\375MMM+W\256\034>|8\026w\303\254\377\346\346\3465k\326\234p\302\t\302\233\034M&SUU\025\327}\225\343X(\345\306\245#\240\245\212\246i\275^_UU\205\216{\201\007C\323\264\313\345\252\256\256\346Z\017`\016*n\211\371\221C\032\215\306f\263\t<\274\001\3410\205\360\216;\356x\363\3157O=\365T\274=\317?\377\374m\267\335\006\000X<\033\272\023F89,\241\027\305\321\2031\003\334\315\303\333Y\020\360\245T*\007\245\253\031\346\"\025\024\027G\216A9\004\000\212\242\006\313\031P,\207j\265\332\355vWTTpC\302\272\332\307`\363\222\201\n[\243(\252\275\275=\221H\\q\305\025\323\246M\0335j\324\262e\313\336}\367]8x\014\364\276}\373\352\352\352\016\346\004G%\344(\353p\211D\"\211D\302\017P\222\311d\032\215\006\367*\250<p[\027.\000zPL\013\320]\264[\241P(\225\312\202\270\252c\241\376l\257a_\203e\347\3

In [19]:
b"\354\236\245\353\271\204\353\266\204\353\245\230 : \353\210\210\354\236\245\354\213\235\nSTR : +21 (6 +15)\nDEX : +72 (6 +66)\nINT : +36 (6 +30)\nLUK : +21 (6 +15)\n\352\263\265\352\262\251\353\240\245 : +1\n\353\247\210\353\240\245 : +1\n\353\260\251\354\226\264\353\240\245 : +100\n\354\227\205\352\267\270\353\240\210\354\235\264\353\223\234 \352\260\200\353\212\245 \355\232\237\354\210\230: 3 (\353\263\265\352\265\254 \352\260\200\353\212\245 \355\232\237\354\210\230: 0)\n".decode('utf-8')

&#39;장비분류 : 눈장식\nSTR : +21 (6 +15)\nDEX : +72 (6 +66)\nINT : +36 (6 +30)\nLUK : +21 (6 +15)\n공격력 : +1\n마력 : +1\n방어력 : +100\n업그레이드 가능 횟수: 3 (복구 가능 횟수: 0)\n&#39;

In [29]:
for raw_record in tf.data.TFRecordDataset("../attention-ocr-original/datasets/testdata/fsns/fsns-00000-of-00001"):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    #print(example.features.feature["image/text"])
    print(example.features)
    break

\351u\236\352\220o\010n\274J\371\206\257\306\207[\304\000\330X\255?^\270\357\363\250\367\346\370\307\031!\274*SD\253\t\031;\013\022w\353\241\201\020\314\255\265\346\346)%g2\&quot;\026\032x D\007\345\211\&#39;\031\245\361\361\351\010j\340\216\206\303\220\336\177x\377/\377\362O\237~\3760\014\0033?\037\236~\371\365\257\207\303\341\247\237\336\377\364\351\303y\236\227?\257\207\323\251\264jh\273\253]P@\315=\206\275\347\363\371\341\341\301\335\307q\004\000\007\337\337\\\237\226\371\351x\030w\303n\314\255E/\030\353)\020b\347wwwwwoj-\265\266q\034?|\370p\377p\177x&gt;\214\303\344\257^\264\273\227\360\215$p 3\347\304\343\270\273\276\332\233\332\351|\256\265\271a\316\0033\347\234\315\274\265\322K-\325$\222\206\\\226\316\325\207\316\216!\&quot;\252\265\212\210\202\366w\313!z\270f]\213\210\021\003\212t\367&lt;\014\210 ,$\254\235l\212)\3134N\327\327\327C\316\347\363y&gt;\236\326ueB\021I\222\240\265\352\336\314H\255\224\352\212\255\252\273mT\254\016\300\306\275\033RJ\270U\216\257G\303\260\221\377\254

In [171]:
record_iterator = tf1.io.tf_record_iterator("data/{}/tfexample_train".format(dataset_name))

for record in record_iterator:
    ex = tf.train.Example()
    ex.ParseFromString(record)
    print(ex)

321\250\323\351p\326)\221H\264Z-\000\350\365\372h4\212\001=\032\215J\245\322x&lt;.\314*\025\335\326\023\211\004vc\316[\220\026\202\236\\\002\014\214C&amp;\223Y\255\326\326\326\326\256\256\256X,\206V\213\330\3212\357#\026rT\277k\006Wi\016\000\230L\246q\343\306\341D\000\2319s\346\336\325\017O\2336\315l6\337\177\377\375\234P\212\032*\267/VZZz\347\235w\356\3358\275^oOO\217\325j5\030\014j\265\372\276\373\356khh@#\224k\256\271\246\271\2719\257\224\247W4\032\315\270q\343\306\215\033WSS\203G\216;\3568\256\370\203\343\322K/\355\265\370\226\257\227+\024\n\214V\231LF\251T\242\335\002\3461qEp8\317\342\317bN\032\263\323\023\312\244p\314\210_\256\3525`1\354\364\247\330\251\226\256/\236\277\025\244R\251V\253u\271\\\023\307\273G\017\t\356\241C\341\221\303a\334a0j\330\366\253C\312\2412\337\262\037N\030\r\325\245{:\222]\201\t\n\330\235\004\000\022\211Dsss&lt;\036/))\301j\033\257\327\033\014\006)5\364W1\270\226\204g\237}\366e\227]\366\301\007\037\\w\335uk\327\256\375\327\277\376\005\000\177\372\323\237^z

In [137]:
image_feature_description = {
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    'image/format': tf.io.FixedLenFeature([], tf.string),
    'image/width': tf.io.FixedLenFeature([], tf.int64),
    'image/orig_width': tf.io.FixedLenFeature([], tf.int64),
    'image/class': tf.io.FixedLenFeature([], tf.int64),
    'image/unpadded_class': tf.io.FixedLenFeature([], tf.int64),
    'image/text': tf.io.FixedLenFeature([], tf.string),
    # 'height': _int64_feature([crop_data.shape[0]]),
}

In [138]:
def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

In [139]:
raw_image_dataset = tf.data.TFRecordDataset("data/{}/tfexample_train".format(dataset_name))

In [140]:
parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

&lt;MapDataset shapes: {image/class: (), image/encoded: (), image/format: (), image/orig_width: (), image/text: (), image/unpadded_class: (), image/width: ()}, types: {image/class: tf.int64, image/encoded: tf.string, image/format: tf.string, image/orig_width: tf.int64, image/text: tf.string, image/unpadded_class: tf.int64, image/width: tf.int64}&gt;

In [141]:
import IPython.display as display
for image_features in parsed_image_dataset:
    #image_raw = image_features['image/encoded'].numpy()
    #display.display(display.Image(data=image_raw))
    print(image_features['image/width'])

InvalidArgumentError: Key: image/class.  Can&#39;t parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]